In [1]:
# !pip install xgboost
# !pip install lightgbm

In [2]:
# 输入数据的参数
_INPUT1='{"name":"input1","type":0,"uri":"tmp_08acd76f74ac46c0b2d644822d282bea"}'
_INPUT2='{"name":"input2","type":0,"uri":"tmp_fbf13fa0fc4e470dab25b537975d561d"}'
_INPUT3='{"name":"input3","type":0,"uri":"tmp_56b0903ca61f432abb87ee2dd4115c29"}'
_INPUT4='{"name":"input4","type":0,"uri":"tmp_488f5dc446454fe78445329009ed531d"}'

# 输出数据的参数
_OUTPUT='[{"name":"output1","type":0,"uri":"tmp_7bb4285b6dfd4090a52303928a59e0c4"},{"name":"output2","type":0,"uri":"tmp_bb421772d288408c9fa024dcd6451e32"},{"name":"output3","type":0,"uri":"tmp_1aeaf77f93ab4e1cad39b7e5f60f29d4"},{"name":"output4","type":0,"uri":"tmp_7d72ccfe4e0a41569ea354447bbe1f29"}]'

# 自定义参数


In [3]:
import numpy as np
import pandas as pd
import wfio
import xgboost as xgb
import random
df_train1 = wfio.read_dataframe(_INPUT1)
df_train2 = wfio.read_dataframe(_INPUT2)
df_testA = wfio.read_dataframe(_INPUT3)
df_testB = wfio.read_dataframe(_INPUT4)

In [4]:
df_train2["yc"]=df_train2["yc"].astype('int')
df_zc = df_train2[df_train2["yc"]==0].reset_index(drop=True)
df_yc = df_train2[df_train2["yc"]==1].reset_index(drop=True)

In [5]:
df_valid = pd.concat([df_zc[:500],df_yc[:500]],axis=0,ignore_index=True)
df_train = pd.concat([df_zc[500:],df_yc[500:]],axis=0,ignore_index=True)

In [6]:
colnames = ['2017q1fphdsl', '2017q1jxje', '2017q1jxsl', '2017q1kpje', '2017q1kps', '2017q1kpse', '2017q1kpsl', '2017q1rkse', '2017q2fphdsl', '2017q2jxje', '2017q2jxsl', '2017q2kpje', '2017q2kps', '2017q2kpse', '2017q2kpsl', '2017q2rkse', '2017q3fphdsl', '2017q3jxje', '2017q3jxsl', '2017q3kpje', '2017q3kps', '2017q3kpse', '2017q3kpsl', '2017q3rkse', '2017q4fphdsl', '2017q4jxje', '2017q4jxsl', '2017q4kpje', '2017q4kps', '2017q4kpse', '2017q4kpsl', '2017q4rkse', '2018q1fphdsl', '2018q1jxje', '2018q1jxsl', '2018q1kpje', '2018q1kps', '2018q1kpse', '2018q1kpsl', '2018q1rkse', '2018q2fphdsl', '2018q2jxje', '2018q2jxsl', '2018q2kpje', '2018q2kps', '2018q2kpse', '2018q2kpsl', '2018q2rkse', '2018q3fphdsl', '2018q3jxje', '2018q3jxsl', '2018q3kpje', '2018q3kps', '2018q3kpse', '2018q3kpsl', '2018q3rkse', '2018q4fphdsl', '2018q4jxje', '2018q4jxsl', '2018q4kpje', '2018q4kps', '2018q4kpse', '2018q4kpsl', '2018q4rkse', '2019q1fphdsl', '2019q1jxje', '2019q1jxsl', '2019q1kpje', '2019q1kps', '2019q1kpse', '2019q1kpsl', '2019q1rkse', '2019q2fphdsl', '2019q2jxje', '2019q2jxsl', '2019q2kpje', '2019q2kps', '2019q2kpse', '2019q2kpsl', '2019q2rkse', '2019q3fphdsl', '2019q3jxje', '2019q3jxsl', '2019q3kpje', '2019q3kps', '2019q3kpse', '2019q3kpsl', '2019q3rkse', '2019q4fphdsl', '2019q4jxje', '2019q4jxsl', '2019q4kpje', '2019q4kps', '2019q4kpse', '2019q4kpsl', '2019q4rkse',\
'bsrxmmp', 'bsrxm',  'cyrs', 'djrq','fddbrxm', 'fdbrxmp', 'xzjd','jyfw', 'nsrmc', 'scjydz', 'hy', 'hydl', 'hyml', 'hyzl', 'yc', 'zcdz', 'zczb', 'zjnsrsbh']
df_train1.columns = colnames
df_testA["yc"] = [0]*len(df_testA)
df_testB["yc"] = [0]*len(df_testB)
df_raw = pd.concat([df_train,df_valid,df_testB],axis=0,sort=False,ignore_index=True)
# df_raw = pd.concat([df,df_testB],axis=0,sort=False,ignore_index=True)
df_raw["yc"] = df_raw["yc"].astype('int')
df_raw["yc"].value_counts()


0    15000
1     1000
Name: yc, dtype: int64

In [7]:
print(df_train.columns)
print(df_raw.columns)

Index(['2017q1fphdsl', '2017q1jxje', '2017q1jxsl', '2017q1kpje', '2017q1kps',
       '2017q1kpse', '2017q1kpsl', '2017q1rkse', '2017q2fphdsl', '2017q2jxje',
       ...
       'hyml', 'hyzl', 'jyfw', 'nsrmc', 'scjydz', 'xzjd', 'yc', 'zcdz', 'zczb',
       'zjnsrsbh'],
      dtype='object', length=114)
Index(['2017q1fphdsl', '2017q1jxje', '2017q1jxsl', '2017q1kpje', '2017q1kps',
       '2017q1kpse', '2017q1kpsl', '2017q1rkse', '2017q2fphdsl', '2017q2jxje',
       ...
       'hyml', 'hyzl', 'jyfw', 'nsrmc', 'scjydz', 'xzjd', 'yc', 'zcdz', 'zczb',
       'zjnsrsbh'],
      dtype='object', length=114)


In [8]:
# 缺失值补充
df_raw.replace('""""""""""""""',np.nan,inplace=True)
df_raw.replace('',np.nan,inplace=True)
median_col = ["zczb","cyrs"]
df_raw.loc[df_raw.cyrs==0,"cyrs"]=df_raw.cyrs.median()
df_raw[median_col] = df_raw[median_col].fillna(value=df_raw[median_col].median())
df_raw.fillna(value=0.0,inplace=True)
df_raw.isnull().sum().sort_values(ascending=False)[:5]

zjnsrsbh        0
2017q4kps       0
2017q4kpsl      0
2017q4rkse      0
2018q1fphdsl    0
dtype: int64

In [9]:
val_colname = df_raw.columns[:96].tolist()
val_colname.extend(["cyrs","zczb"]) # 从业人数, 注册资本
for ele in val_colname:
    df_raw[ele] = df_raw[ele].astype('float')

In [10]:
############################### 对抗验证 LGB ############################################
%matplotlib inline
import xgboost as xgb
from xgboost import DMatrix
import lightgbm as lgb
from lightgbm import Dataset
from sklearn.model_selection import KFold,StratifiedKFold, GridSearchCV,cross_val_score
from sklearn.feature_selection import SelectFromModel

params = {'num_leaves': 60, #结果对最终效果影响较大，越大值越好，太大会出现过拟合
          'min_data_in_leaf': 30,
          'objective': 'binary', #定义的目标函数
          'max_depth': -1,
          'min_child_samples': 100,
          'learning_rate': 0.1,
          "min_sum_hessian_in_leaf": 6,
          "boosting": "gbdt",
          "feature_fraction": 0.9,  #提取的特征比率
          "bagging_freq": 1,
          'subsample': 1.0,
          "bagging_fraction": 0.8,
          "bagging_seed": 11,
          "lambda_l1": 0.1,             #l1正则
#           'lambda_l2': 0.001,     #l2正则
          "verbosity": -1,
          "nthread": 8,                #线程数量，-1表示全部线程，线程越多，运行的速度越快
          'metric': {'auc'},  ##评价函数选择
          "random_state": 33, #随机数种子，可以防止每次运行的结果不一致
          'scale_pos_weight':1, # 负样本数0/正样本数1, 如果取值大于0的话，在类别样本不平衡的情况下有助于快速收敛。
          # 'device': 'gpu' ##如果安装的事gpu版本的lightgbm,可以加快运算
          }

"""仅加入98个数值特征"""
df_val = df_raw[val_colname].copy() # 98
# df_val = df.drop(df.columns[:96],axis=1).copy() # 131
print(len(df_val.columns))
X = df_val.values
trian_num = 10000
y = np.zeros(len(X))
y[trian_num:] = 1
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=66)
cv_weight = []
cv_index = []
for fold_n, (train_index, valid_index) in enumerate(folds.split(X,y)):
    X_train, X_valid = X[train_index], X[valid_index]
    y_train, y_valid = y[train_index], y[valid_index]
    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_valid, label=y_valid)
    model = lgb.train(params=params,train_set=train_data,num_boost_round=500,
                      valid_sets = [train_data, valid_data],verbose_eval=50,early_stopping_rounds=50)
    pred = model.predict(X_valid, num_iteration=model.best_iteration)
    cv_index.append(valid_index)
    cv_weight.append(pred)
    
cv_index = np.concatenate(cv_index,axis=0)
cv_weight = np.concatenate(cv_weight,axis=0)
weight = pd.Series(cv_weight,index=cv_index)
weight = weight.sort_index()
print(weight.head())
weight_train = weight[weight.index<trian_num]
# weight_train = weight.copy()
print(len(weight_train))
print(weight_train.sort_values(ascending=False)[:5])
print(weight_train.head())


98
Training until validation scores don't improve for 50 rounds
[50]	training's auc: 0.885287	valid_1's auc: 0.686156
Early stopping, best iteration is:
[6]	training's auc: 0.794042	valid_1's auc: 0.688319
Training until validation scores don't improve for 50 rounds
[50]	training's auc: 0.884297	valid_1's auc: 0.693432
Early stopping, best iteration is:
[20]	training's auc: 0.838072	valid_1's auc: 0.699022
Training until validation scores don't improve for 50 rounds
[50]	training's auc: 0.886315	valid_1's auc: 0.693065
Early stopping, best iteration is:
[41]	training's auc: 0.874749	valid_1's auc: 0.693828
Training until validation scores don't improve for 50 rounds
[50]	training's auc: 0.885651	valid_1's auc: 0.703136
Early stopping, best iteration is:
[2]	training's auc: 0.760196	valid_1's auc: 0.706662
Training until validation scores don't improve for 50 rounds
[50]	training's auc: 0.884978	valid_1's auc: 0.694278
[100]	training's auc: 0.922785	valid_1's auc: 0.695751
Early stoppin

In [11]:
"""仅加入98个数值特征"""
df = df_raw[val_colname].copy() # 98
print("before:\n",df.isnull().sum().sort_values(ascending=False)[:5])
""" 加入其他特征 """
Year_Quarter = ["2017q1","2017q2","2017q3","2017q4","2018q1","2018q2","2018q3","2018q4","2019q1","2019q2","2019q3","2019q4"]
extend_valfeat = []

# zczbccyrs 注册资本/从业人数
# df["zczbccyrs"] = df["zczb"]/df["cyrs"]
# extend_valfeat.append("zczbccyrs")

# FPDIFF 发票核定数量差值 
for time_prefix in Year_Quarter:
    FPHDSL = time_prefix+'fphdsl' # 发票核定数量
    KPS = time_prefix+'kps' # 开票数
    FPDIFF = time_prefix + 'fpdiff' 
    fp_diff = df[FPHDSL]-df[KPS] # 差值>0
    extend_valfeat.append(FPDIFF)
    df[FPDIFF] = fp_diff

# JXSE 进项税额=进项金额*进项税率 
for time_prefix in Year_Quarter:
    JXJE = time_prefix + 'jxje'
    JXSL = time_prefix + 'jxsl'
    JXSE = time_prefix + 'jxse'
    df.loc[df[JXSL]<=0.03,JXSL]=0
    jxse = df[JXJE]*df[JXSL]
    extend_valfeat.append(JXSE)
    df[JXSE] = jxse

# CALRKSE 计算的入库税额
# SEDIFF 税额差值=(计算的入库税额-入库税额)*I(入库税额>0) 
# JXYC 进销异常，查看是否存在有进无销或有销无进 (xor)
for time_prefix in Year_Quarter:
    # CALRKSE
    cal_rkse = df[time_prefix+'kpse']-df[time_prefix+'jxse'] # 销项 - 进项
    CALRKSE = time_prefix+'calrkse'
    extend_valfeat.append(CALRKSE)
    df[CALRKSE] = cal_rkse
    # SEDIFF
    se_diff = (df[time_prefix+'rkse']-cal_rkse)*(df[time_prefix+'rkse']>0)
    SEDIFF = time_prefix+'sediff'
    extend_valfeat.append(SEDIFF)
    df[SEDIFF] = se_diff
    # JXYC
    has_kpje = df[time_prefix+'kpje'] > 0
    has_jxje = df[time_prefix+'jxje'] > 0
    JXYC = time_prefix+'jxyc'
    extend_valfeat.append(JXYC)
    df[JXYC] = has_kpje ^ has_jxje

    
"""一些特征的统计值 max,min,mean,std"""
# 发票核定数量 FPHDSL max,min,mean,std
# ...
def Stat_fea(feat):
    global df
    feat_vals = []
    for time_prefix in Year_Quarter:
        colname = time_prefix+feat
        feat_vals.append(df[colname].values)
    feat_all = []
    kpcs = []
    for i in range(len(df)):
        feat_line = []
        for t in range(len(feat_vals)):
            now_val = feat_vals[t][i]
            if(abs(now_val) < 1e-8): # 特征值为0
                continue
            feat_line.append(now_val)
        feat_line = np.asarray(feat_line)
        if(len(feat_line) == 0):
            feat_all.append([0,0,0,0,0])
            kpcs.append(0)
        else:
            feat_all.append([feat_line.max(),feat_line.min(),feat_line.mean(),feat_line.std(),feat_line.sum()])
            kpcs.append(len(feat_line))
    feat_all = np.asarray(feat_all)
    feat_names = [feat+'MAX',feat+'MIN',feat+'MEAN',feat+'STD',feat+'SUM']
    extend_valfeat.extend(feat_names)
    feat_df = pd.DataFrame(feat_all,columns=feat_names)
    df = pd.concat([df,feat_df],axis=1)
    if(feat == 'fphdsl'):
        extend_valfeat.append("kpcs")
        df["kpcs"] = kpcs
    
Stat_fea('fphdsl')
Stat_fea('kpje')
Stat_fea('jxje') # 强特
Stat_fea('rkse') 
Stat_fea('fpdiff')
Stat_fea('calrkse')
Stat_fea('sediff')

"""增幅,增幅量/每季度,变化率的max,min,mean,std"""
# 发票核定数量 FPHDSL 
# 开票金额及 KPJE
# 进项金额及 JXJE
def ZengFu(feat): # max,min,mean,std
    global df
    feat_vals = []
    for time_prefix in Year_Quarter:
        colname = time_prefix+feat
        feat_vals.append(df[colname].values)
    zf_all = []
    ratio_all = []
    speed_all = []
    for i in range(len(df)):
        zf_line = []
        ratio_line = []
        speed_line = []
        pre = 0
        last_time = 0
        for t in range(len(feat_vals)):
            now_val = feat_vals[t][i]
            if(abs(now_val) < 1e-8): # 特征值为0
                continue
            if(abs(pre) > 1e-8):
                zf_line.append(now_val - pre)
                ratio_line.append((now_val - pre)/pre)
                speed_line.append((now_val - pre)/(t-last_time))
            pre = now_val
            last_time = t
        zf_line = np.asarray(zf_line)
        ratio_line = np.asarray(ratio_line)
        speed_line = np.asarray(speed_line)
        if(len(zf_line) == 0):
            zf_all.append([0,0,0,0])
            ratio_all.append([0,0,0,0])
            speed_all.append([0,0,0,0])
        else:
            zf_all.append([zf_line.max(),zf_line.min(),zf_line.mean(),zf_line.std()])
            ratio_all.append([ratio_line.max(),ratio_line.min(),ratio_line.mean(),ratio_line.std()])
            speed_all.append([speed_line.max(),speed_line.min(),speed_line.mean(),speed_line.std()])
    zf_all = np.asarray(zf_all)
    ratio_all = np.asarray(ratio_all)
    speed_all = np.asarray(speed_all)
    zf_names = [feat+'ZFMAX',feat+'ZFMIN',feat+'ZFMEAN',feat+'ZFSTD']
    ratio_names = [feat+'ZFRMAX',feat+'ZFRMIN',feat+'ZFRMEAN',feat+'ZFRSTD']
    speed_names = [feat+'ZFSPMAX',feat+'ZFSPMIN',feat+'ZFSPMEAN',feat+'ZFSPSTD']
    extend_valfeat.extend(zf_names)
    extend_valfeat.extend(ratio_names)
    extend_valfeat.extend(speed_names)
    zf_df = pd.DataFrame(zf_all,columns=zf_names)
    ratio_df = pd.DataFrame(ratio_all,columns=ratio_names)
    speed_df = pd.DataFrame(speed_all,columns=speed_names)
    df = pd.concat([df,zf_df,ratio_df,speed_df],axis=1)
    
ZengFu('fphdsl')
ZengFu('kpje')
ZengFu('jxje')

"""第一次的值，开业时长（月），值/开业时长（每月）"""
# 第一次发票核定数量 FPHDSL
...
def First_Month(feat):
    global df,df_raw
    feat_vals = []
    for time_prefix in Year_Quarter:
        colname = time_prefix+feat
        feat_vals.append(df[colname].values)
    time = pd.to_datetime(df_raw["djrq"])
    year,month,day = time.dt.year, time.dt.month, time.dt.day
    first_val = []
    for i in range(len(df)):
        row_val = [36,0,0]
        for t in range(len(feat_vals)):
            now_val = feat_vals[t][i]
            if(abs(now_val) > 0): # 第一次值不为0
# =============== method1 - 94.567 ===================
                if(year[i]>2016 or i<10000):
                    dmonth = (2017-(year[i]-1))*12 + (12-month[i]) + (t+1)*3
                else:
                    dmonth = (2017-(year[i]+1))*12 + (12-month[i]) + (t+1)*3
                    
# =============== method2 - 94.277 ===================
#                 dmonth = (2017-(year[i]-1))*12 + (12-month[i]) + (t+1)*3
                
# =============== method3 - 93.773 ===================
#                 if(i<10000):
#                     dmonth = (2017-(year[i]-1))*12 + (12-month[i]) + (t+1)*3
#                 else:
#                     if(year[i]*12+month[i] < 2017*12+(t+1)*3 or (year[i]*12+month[i] <= 2017*12+(t+1)*3 and day[i] < 15)):
#                         dmonth = 2017*12+(t+1)*3 - (year[i]*12+month[i])
#                     else:
#                         dmonth = (2017-(year[i]-1))*12 + (12-month[i]) + (t+1)*3
                if(day[i] < 15):
                    dmonth += 1
                row_val = [dmonth,now_val,now_val/dmonth]
                break
        first_val.append(row_val)
    first_val = np.asarray(first_val)
    fv_colname = [feat+"MONTH",feat+"FVAL",feat+"FVALMEAN"] 
    fv_df = pd.DataFrame(first_val,columns=fv_colname)
    if(feat == "fphdsl"):
        extend_valfeat.extend(fv_colname)
    else:
        extend_valfeat.extend(fv_colname[1:])
        fv_df.drop(feat+"MONTH",axis=1,inplace=True)
    df = pd.concat([df,fv_df],axis=1)
    
print("mid:\n",df.isnull().sum().sort_values(ascending=False)[:5])    
First_Month('fphdsl')
First_Month('kps')
First_Month('fpdiff')
First_Month('kpje')
First_Month('jxje') #strong
First_Month('rkse') #strong
First_Month('calrkse')
First_Month('sediff')
print("after:\n",df.isnull().sum().sort_values(ascending=False)[:5])

before:
 zczb            0
2017q4fphdsl    0
2017q4jxsl      0
2017q4kpje      0
2017q4kps       0
dtype: int64
mid:
 jxjeZFSPSTD    0
2019q3kpse     0
2019q3kpje     0
2019q3jxsl     0
2019q3jxje     0
dtype: int64
after:
 sediffFVALMEAN    0
2019q4kpje        0
2019q4jxje        0
2019q4fphdsl      0
2019q3rkse        0
dtype: int64


In [12]:
# PCA处理类别特征
# from sklearn.decomposition import PCA
# # 加入数值和类别特征(转化为one-hot)
# # cate_colname = ["XZJD","HY","HYDL","HYML","HYZL"]
# cate_colname = ["hy"]
# df_cate = df_raw[cate_colname].copy()

# train_num = 20000
# for colname in cate_colname:
#     cate_train = set(df_cate[colname][:train_num])
# #     print(cate_train)
#     col_series = df_cate[colname].values.copy()
#     for i in range(train_num,len(df_cate)): # test dataset
#         if(col_series[i] not in cate_train):
#             col_series[i] = 'UNK'
#     fea_df = pd.get_dummies(col_series)
# #     print(sum(fea_df.sum(axis=1)))
#     pca=PCA(n_components=0.9)
#     newfea=pca.fit_transform(fea_df)
# #     print(newfea_df.shape)
#     pca_col = []
#     for i in range(newfea.shape[1]):
#         pca_col.append("pca_{}".format(i))
#     newfea_df = pd.DataFrame(newfea,columns=pca_col)
#     df_cate = pd.concat([df_cate,newfea_df],axis=1)
#     df_cate.drop(colname,axis=1,inplace=True)
# df = pd.concat([df,df_cate],axis=1) 
# # col_name = df_cate.columns.tolist()
# # used_name = set()
# # for i,name in enumerate(col_name):
# #     if(name in used_name):
# #         col_name[i] = name+str(i)
# #     else:
# #         used_name.add(name)
# # print(pca.explained_variance_ratio_)

In [13]:
# 归一化
df_norm = df.copy()
mx = df_norm.max()
mx[mx.abs()<1e-8] = 1e-8
df_norm = (df_norm-df_norm.min())/(mx-df_norm.min())

In [14]:
import numpy as np
from sklearn.model_selection import KFold
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier,GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC,SVC

# # custom eval_metric
# def weight_ce_lgb(y, pred):
# #     score = (1-np.abs(1/np.sum(weight)*np.sum((y*np.log(pred) + (1-y)*np.log(1-pred))*weight)))*100 # 带权重的交叉熵
#     score = -1/len(y)*np.sum(0.9*y*np.log(pred) + 0.1*(1-y)*np.log(1-pred))
#     return 'Weight_CE', score, False

# # custom eval_metric
# def weight_ce_xgb(y,pred):
# #     y = dtrain.get_label()
# #     weight = dtrain.get_weight()
#     score = -1/len(y)*np.sum(0.9*y*np.log(pred) + 0.1*(1-y)*np.log(1-pred))
# #     score = -1/np.sum(weight)*np.sum((y*np.log(predt) + (1-y)*np.log(1-predt))*weight) # 带权重的交叉熵
#     # score = (1-np.abs(1/np.sum(weight)*np.sum((y*np.log(predt) + (1-y)*np.log(1-predt))*weight)))*100 # 带权重的交叉熵
#     return 'Weight_CE', score


# 模型预测
def score_func(y,pred):
    yc_num = int(len(y)*0.5)
    acc = (1 - np.abs((1/yc_num)*np.sum(0.5*y*np.log(pred) + 0.5*(1-y)*np.log(1-pred))))*100
    return acc

# stacking
def get_stacking(clf_tp,train_set, val_set,weight=None,verbose=100):
    model_name,clf = clf_tp[0],clf_tp[1]
    X_train, y_train = train_set[0], train_set[1]
    X_val, y_val =  val_set[0], val_set[1]
    train_num = X_train.shape[0]
    if(model_name in ['lgb_gbdt','lgb_rf']):
        clf.fit(X=X_train,y=y_train,sample_weight=weight,eval_set=[(X_val,y_val)],\
                early_stopping_rounds=50,verbose=verbose)
    elif(model_name == 'xgb'):
        clf.fit(X=X_train,y=y_train,sample_weight=weight,eval_set=[(X_val,y_val)],\
                early_stopping_rounds=50,verbose=verbose,eval_metric='logloss')
    elif(model_name == 'nn'):
        clf.fit(X_train, y_train)
    else:
        clf.fit(X_train, y_train, sample_weight=weight) # 模型训练
    second_level_train_set = clf.predict_proba(X_val)[:,1] 
 
    return second_level_train_set, clf

drop_col = []
for ele in df.columns:
    if(str.isdigit(ele[:4])):
        drop_col.append(ele)
# df_drop = df.drop(drop_col,axis=1) # drop掉12个季度的数值特征
df_drop = df_norm.drop(drop_col,axis=1) # drop掉12个季度的数值特征
# df_drop = df[val_colname]
"""训练集"""
train_num = 9000
train_X = df_drop[:train_num].values
train_y = df_raw["yc"][:train_num].values
print('num of train_set:',train_X.shape[0])
print('num of feat:',train_X.shape[1])
train_dataset = (train_X,train_y)

"""验证集"""
valid_X = df_drop[train_num:10000].values
valid_y = df_raw["yc"][train_num:10000].values
print('num of valid_set:',valid_X.shape[0])
val_dataset = (valid_X,valid_y)

"""测试集"""
test_X = df_drop[10000:].values
print('num of test_set:',test_X.shape[0])

"""模型选择"""
num_round = 200
# lgb_gbdt = lgb.LGBMClassifier(boosting_type='gbdt',objective = 'binary' ,n_estimators = num_round, learning_rate = 0.1,max_depth=-1,\
#                          reg_alpha=0.1,num_leaves = 35,feature_fraction=0.9,bagging_fraction= 0.9,bagging_freq= 8,subsample = 0.7,\
#                          lambda_l1= 0.6,lambda_l2= 0,n_jobs = 8,random_state=0)
# lgb_rf = lgb.LGBMClassifier(boosting_type='rf',objective = 'binary',n_estimators = num_round, learning_rate = 0.1,\
#                          num_leaves = 60,feature_fraction=0.7,bagging_fraction= 0.9,bagging_freq= 8,subsample = 0.7,\
#                          lambda_l1= 0.6,lambda_l2= 0,n_jobs = 8,random_state=33)
# xgb_model = xgb.XGBClassifier(n_estimators=num_round,max_depth=3,learning_rate=2.5,gamma=0,min_child_weight=1,max_delta_step=2,\
#                         subsample=0.75,colsample_bytree=0.7,reg_alpha=0,reg_lambda=0.6,scale_pos_weight=1,n_jobs=8,random_state=30)
# svc = SVC(C=100,probability=True)
# lr = LogisticRegression(max_iter=num_round)
# nn = MLPClassifier(solver='adam',activation = 'relu',max_iter = num_round,alpha = 1e-5,learning_rate_init=0.001,validation_fraction=0.1,\
#                    hidden_layer_sizes = (64,64),random_state = 1,verbose = 50,early_stopping=True,n_iter_no_change=50)
# model_sets = [("xgb",xgb_model),("lgb_gbdt",lgb_gbdt),("nn",nn),("rf",rf_model)]
# model_sets = [("xgb",xgb_model)]
"""底层分类器"""
model_sets = []
for seed in range(200,400):
    xgb_model = xgb.XGBClassifier(n_estimators=num_round,max_depth=3,learning_rate=2.5,gamma=0,min_child_weight=1,max_delta_step=2,\
                        subsample=0.75,colsample_bytree=0.7,reg_alpha=0,reg_lambda=0.6,scale_pos_weight=1,n_jobs=8,random_state=seed)
    model_sets.append(("xgb",xgb_model))
train_predsets = []
model_train = []
weight = weight_train.values
for i,clf_tp in enumerate(model_sets):
#     train_pred, clf = get_stacking(clf_tp,train_dataset,val_dataset,weight=np.ones(train_num))
    train_pred, clf = get_stacking(clf_tp,train_dataset,val_dataset,weight=weight[:train_num])
    pred = clf.predict_proba(valid_X)[:,1]
    acc = score_func(valid_y,pred)
    print("-- {} best acc: {}\n".format(clf_tp[0],acc))
    train_predsets.append(train_pred) # [M,N] - N为样本数,M为模型数
    model_train.append(clf)
meta_train = np.concatenate([result_set.reshape(-1,1) for result_set in train_predsets], axis=1) #meta feature (N,M)


num of train_set: 9000
num of feat: 91
num of valid_set: 1000
num of test_set: 6000
[0]	validation_0-logloss:0.101462
Will train until validation_0-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[1]	validation_0-logloss:0.044724

-- xgb best acc: 95.5276012184011

[0]	validation_0-logloss:0.101457
Will train until validation_0-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[1]	validation_0-logloss:0.048279

-- xgb best acc: 95.17212392003276

[0]	validation_0-logloss:0.10128
Will train until validation_0-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[1]	validation_0-logloss:0.041723

-- xgb best acc: 95.82773717918026

[0]	validation_0-logloss:0.101359
Will train until validation_0-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[3]	validation_0-logloss:0.043138

-- xgb best acc: 95.68623955762014

[0]	validation_0-logloss:0.101398
Will train until validation_0-logloss hasn't improved in 50 rounds.
Stopping. Best itera

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:29: RuntimeWarning: divide by zero encountered in log
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in multiply


Stopping. Best iteration:
[1]	validation_0-logloss:0.042703

-- xgb best acc: 95.7296930843353

[0]	validation_0-logloss:0.101424
Will train until validation_0-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[1]	validation_0-logloss:0.046115

-- xgb best acc: 95.38848399216803

[0]	validation_0-logloss:0.1014
Will train until validation_0-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[3]	validation_0-logloss:0.040829

-- xgb best acc: 95.91714114690548

[0]	validation_0-logloss:0.101358
Will train until validation_0-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[1]	validation_0-logloss:0.042787

-- xgb best acc: 95.72126569985994

[0]	validation_0-logloss:0.101305
Will train until validation_0-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[1]	validation_0-logloss:0.041912

-- xgb best acc: 95.80880687877507

[0]	validation_0-logloss:0.101391
Will train until validation_0-logloss hasn't improved in 50 rounds.
Stopping

In [15]:
""" 元分类器 """
# ==================== sklearn clf ========================
meta_clf = LogisticRegression(max_iter=num_round,C=25,penalty='l2',solver='sag',warm_start=True) # 逻辑回归 89.198
# meta_clf = MLPClassifier(solver='adam',activation = 'logistic',max_iter = 200,alpha = 1e-5,learning_rate_init=0.001,validation_fraction=0.1,\
#                    hidden_layer_sizes = (100),random_state = 1,verbose = 50)
# meta_clf = RandomForestClassifier(verbose=1) # 随机森林 
# meta_clf = SVC(C=3,probability=True) # SVM # 84.96216
# meta_clf = AdaBoostClassifier() # 65.692
meta_clf.fit(meta_train, valid_y)

# ===================== other clf ==========================
# num_round_meta = 1000
# meta_clf = lgb.LGBMClassifier(boosting_type='gbdt',objective = 'binary' ,n_estimators = num_round_meta, learning_rate = 0.01,max_depth=6,\
#                          reg_alpha=0.1,num_leaves = 35,feature_fraction=0.9,bagging_fraction= 0.9,bagging_freq= 8,subsample = 0.7,\
#                          lambda_l1= 0.6,lambda_l2= 0,n_jobs = 8,random_state=33)
# lgb_rf = lgb.LGBMClassifier(boosting_type='rf',objective = 'binary',n_estimators = num_round, learning_rate = 0.01,\
#                          num_leaves = 35,feature_fraction=0.7,bagging_fraction= 0.9,bagging_freq= 8,subsample = 0.7,\
#                          lambda_l1= 0.6,lambda_l2= 0,n_jobs = 8,random_state=34)
# xgb_model = xgb.XGBClassifier(max_depth=3, learning_rate=0.1,n_estimators=num_round,n_jobs=8,gamma=0.1,reg_alpha=0,reg_lambda=2,\
#                               colsample_bytree=0.7,min_child_weight=3, subsample=0.7,random_state=35)

# meta_clf.fit(X=meta_train,y=train_y,sample_weight=weight,eval_set=[(test_X,test_y)],\
#                 early_stopping_rounds=100,verbose=100)

#计时打分程序
from ustciscrBDL_B import get_score
get_score.post_user_id('19')
test_sets = []
for clf in model_train:
    pred = clf.predict_proba(test_X)[:,1]
    test_sets.append(pred)
meta_test = np.concatenate([y_test_set.reshape(-1,1) for y_test_set in test_sets], axis=1) # (N, M)
final_pred = meta_clf.predict_proba(meta_test)[:,1]
# final_pred = meta_test.mean(axis=-1)
user_verify_data = pd.DataFrame({'zjnsrsbh':df_raw['zjnsrsbh'][10000:],'Probability':final_pred})
get_score.post_verify_data(user_verify_data)


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


User-id:19
程序开始运行时间为：2019-12-22 12:12:50.677992
程序结束运行时间为：2019-12-22 12:12:51.325736
程序运行时间（去除打分耗时）为：0.647744
程序的准确率为：95.57256964
最终得分为：96.01531267


In [16]:
# import numpy as np
# from sklearn.model_selection import KFold
# import lightgbm as lgb
# import xgboost as xgb
# from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier,GradientBoostingClassifier, ExtraTreesClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.svm import LinearSVC,SVC

# # # custom eval_metric
# # def weight_ce_lgb(y, pred):
# # #     score = (1-np.abs(1/np.sum(weight)*np.sum((y*np.log(pred) + (1-y)*np.log(1-pred))*weight)))*100 # 带权重的交叉熵
# #     score = -1/len(y)*np.sum(0.9*y*np.log(pred) + 0.1*(1-y)*np.log(1-pred))
# #     return 'Weight_CE', score, False

# # # custom eval_metric
# # def weight_ce_xgb(y,pred):
# # #     y = dtrain.get_label()
# # #     weight = dtrain.get_weight()
# #     score = -1/len(y)*np.sum(0.9*y*np.log(pred) + 0.1*(1-y)*np.log(1-pred))
# # #     score = -1/np.sum(weight)*np.sum((y*np.log(predt) + (1-y)*np.log(1-predt))*weight) # 带权重的交叉熵
# #     # score = (1-np.abs(1/np.sum(weight)*np.sum((y*np.log(predt) + (1-y)*np.log(1-predt))*weight)))*100 # 带权重的交叉熵
# #     return 'Weight_CE', score

# def get_stacking(clf, model_name, x_train, y_train, kf, weight=None):
#     """
#     这个函数是stacking的核心，使用交叉验证的方法得到次级训练集
#     x_train, y_train 的值应该为numpy里面的数组类型 numpy.ndarray .
#     """
#     train_num = x_train.shape[0]
#     second_level_train_set = np.zeros((train_num,))
#     models = []
#     for i,(train_index, test_index) in enumerate(kf.split(x_train,y_train)):
#         x_tra, y_tra = x_train[train_index], y_train[train_index]
#         x_tst, y_tst =  x_train[test_index], y_train[test_index]
        
#         if(model_name in ['lgb_gbdt','lgb_rf']):
#             clf.fit(X=train_X,y=train_y,sample_weight=weight,eval_set=[(x_tst,y_tst)],\
#                     early_stopping_rounds=50,verbose=100)
#         elif(model_name == 'xgb'):
#             clf.fit(X=train_X,y=train_y,sample_weight=weight,eval_set=[(x_tst,y_tst)],early_stopping_rounds=50,verbose=100,eval_metric='logloss')
#         else:
#             clf.fit(x_tra, y_tra, sample_weight=weight[train_index]) # 模型训练
#         models.append(clf)
#         second_level_train_set[test_index] = clf.predict_proba(x_tst)[:,1] # 训练集预测概率,k折后拼成完整
# #             second_level_train_set[test_index] = clf.predict(x_tst)
#         # test_nfolds_sets[:,i] = clf.predict_proba(x_test)[:,1] # k折每次都预测测试集
 
#     return second_level_train_set, models



# num_round = 2000 
# lgb_gbdt = lgb.LGBMClassifier(boosting_type='gbdt',objective = 'binary' ,n_estimators = num_round, learning_rate = 0.01,max_depth=6,\
#                          reg_alpha=0.1,num_leaves = 35,feature_fraction=0.9,bagging_fraction= 0.9,bagging_freq= 8,subsample = 0.7,\
#                          lambda_l1= 0.6,lambda_l2= 0,n_jobs = 8,random_state=33)
# lgb_rf = lgb.LGBMClassifier(boosting_type='rf',objective = 'binary',n_estimators = num_round, learning_rate = 0.01,\
#                          num_leaves = 35,feature_fraction=0.7,bagging_fraction= 0.9,bagging_freq= 8,subsample = 0.7,\
#                          lambda_l1= 0.6,lambda_l2= 0,n_jobs = 8,random_state=34)
# xgb_model = xgb.XGBClassifier(max_depth=3, learning_rate=0.1,n_estimators=num_round,n_jobs=8,gamma=0.1,reg_alpha=0,reg_lambda=2,\
#                               colsample_bytree=0.7,min_child_weight=3, subsample=0.7,random_state=35)
# # svc = LinearSVC()
# rf_model = RandomForestClassifier(verbose=1)
# # adb_model = AdaBoostClassifier()
# # gdbc_model = GradientBoostingClassifier()
# # et_model = ExtraTreesClassifier()

# drop_col = []
# for ele in df.columns:
#     if(str.isdigit(ele[:4])):
#         drop_col.append(ele)
# df_drop = df.drop(drop_col,axis=1) # drop掉12个季度的数值特征
# # df_drop = df[val_colname]
# """训练集"""
# train_num = 10000
# train_X = df_drop[:train_num].values
# train_y = df_raw["yc"][:train_num].values
# print('num of train_set:',train_X.shape[0])
# print('num of feat:',train_X.shape[1])

# """测试集"""
# test_X = df_drop[train_num:].values
# test_y = df_raw["yc"][train_num:].values
# train_sets = []
# n_folds = 5
# kf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=33)
# # model_sets = [xgb_model,lgb_gbdt,rf_model]
# # model_name = ['xgb','lgb_gbdt','rf_model']
# model_sets = [rf_model]
# model_name = ['rf_model']
# model_train = []
# for i,clf in enumerate(model_sets):
#     train_set, models = get_stacking(clf, model_name[i], train_X, train_y, kf, weight=np.ones(train_num))
# #     train_set, models = get_stacking(clf, model_name[i], train_X, train_y, kf, weight=weight_train)
#     train_sets.append(train_set) # [M,N1] - N1为样本数,M为模型数
#     model_train.append(models)

# meta_train = np.concatenate([result_set.reshape(-1,1) for result_set in train_sets], axis=1)

# #使用逻辑回归作为我们的次级分类器
# lr = LogisticRegression(max_iter=num_round)
# lr.fit(meta_train, train_y)



In [17]:
# def predict_prob_stacking(text_X, trained_model,clf,n_folds):
#     test_sets = []
#     for model in trained_model:
#         model_res = np.zeros((len(test_X), n_folds))
#         for i,sub_model in enumerate(model):
#             pred = sub_model.predict_proba(test_X)[:,1]
#             model_res[:,i] = pred
#         res = np.mean(model_res,axis=1)
#         test_sets.append(res)
#     meta_test = np.concatenate([y_test_set.reshape(-1,1) for y_test_set in test_sets], axis=1)
#     y_pred = clf.predict_proba(meta_test)[:,1]
#     return y_pred
    
# def predict_prob_single(test_X, trained_model_single,n_folds):
#     test_sets = []
#     model_res = np.zeros((len(test_X), n_folds))
#     for i,sub_model in enumerate(trained_model_single):
#         pred = sub_model.predict_proba(test_X)[:,1]
#         model_res[:,i] = pred
#     y_pred = np.mean(model_res,axis=1)
#     return y_pred
    
# #计时打分程序
# from ustciscrLab_A import get_score
# get_score.post_user_id('23')
# #====================================模型预测===================
# # print(model_train[0])
# # y_pred = predict_prob_stacking(test_X, model_train,lr,n_folds)
# y_pred = predict_prob_single(test_X, model_train[0],n_folds)
# #=====================================================
# user_verify_data = pd.DataFrame({'zjnsrsbh':df_raw['zjnsrsbh'][train_num:],'Probability':y_pred})
# get_score.post_verify_data(user_verify_data)

In [18]:
# #计时打分程序
# from ustciscrLab_A import get_score
# get_score.post_user_id('23')
# # 模型预测
# test_sets = []
# for model in model_train:
#     model_res = np.zeros((len(test_X), n_folds))
#     for i,sub_model in enumerate(model):
#         pred = sub_model.predict_proba(test_X)[:,1]
#         model_res[:,i] = pred
#     res = np.mean(model_res,axis=1)
#     test_sets.append(res)
# meta_test = np.concatenate([y_test_set.reshape(-1,1) for y_test_set in test_sets], axis=1)
# y_pred = lr.predict_proba(meta_test)[:,1]
# user_verify_data = pd.DataFrame({'zjnsrsbh':df_raw['zjnsrsbh'][train_num:],'Probability':y_pred})
# get_score.post_verify_data(user_verify_data)